In [1]:
import numpy as np
import torch

from train import TrainerDeepSVDD
from preprocess import get_mnist

In [2]:
class Args:

    num_epochs=150
    num_epochs_ae=150
    patience=50
    lr=1e-4
    weight_decay=0.5e-6
    weight_decay_ae=0.5e-3
    lr_ae=1e-4
    lr_milestones=[50]
    batch_size=200
    pretrain=True
    latent_dim=True
    normal_class=3
    
    
args = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = get_mnist(args)

deep_SVDD = TrainerDeepSVDD(args, data, device)

if args.pretrain:
    deep_SVDD.pretrain()

6131/6131: [==============================>.] - ETA 0.3ss
Pretraining Autoencoder... Epoch: 0, Loss: 144.933
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 1, Loss: 112.835
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 2, Loss: 92.621
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 3, Loss: 79.657
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 4, Loss: 70.741
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 5, Loss: 64.212
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 6, Loss: 59.539
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 7, Loss: 55.931
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 8, Loss: 53.061
6131/6131: [======================

6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 76, Loss: 30.338
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 77, Loss: 30.329
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 78, Loss: 30.280
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 79, Loss: 30.256
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 80, Loss: 30.238
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 81, Loss: 30.203
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 82, Loss: 30.171
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 83, Loss: 30.145
6131/6131: [==============================>.] - ETA 0.0s
Pretraining Autoencoder... Epoch: 84, Loss: 30.127
6131/6131: [================

In [3]:
deep_SVDD.train()

6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 0, Loss: 0.905
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 1, Loss: 0.412
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 2, Loss: 0.214
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 3, Loss: 0.125
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 4, Loss: 0.082
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 5, Loss: 0.060
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 6, Loss: 0.047
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 7, Loss: 0.040
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 8, Loss: 0.034
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 9, Lo

6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 80, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 81, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 82, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 83, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 84, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 85, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 86, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 87, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epoch: 88, Loss: 0.003
6131/6131: [==============================>.] - ETA 0.0s
Training Deep SVDD... Epo

In [4]:
net = deep_SVDD.net
c = deep_SVDD.c
scores = []
labels = []

net.eval()
with torch.no_grad():
    for x, y in deep_SVDD.test_loader:
        x = x.float().to(device)
        z = net(x)
        dist = torch.sum((z - c) ** 2, dim=1)
        scores.append(dist.detach().cpu())
        labels.append(y.cpu())

In [5]:
scores = torch.cat(scores).numpy()
labels = torch.cat(labels).numpy()

In [6]:
import pandas as pd 
import matplotlib.pyplot as plt


scores_in = scores[np.where(labels==args.normal_class)[0]]
scores_out = scores[np.where(labels!=args.normal_class)[0]]


in_ = pd.DataFrame(scores_in, columns=['Inlier'])
out_ = pd.DataFrame(scores_out, columns=['Outlier'])


fig, ax = plt.subplots()
in_.plot.kde(ax=ax, legend=True, title='Outliers vs Inliers (Deep SVDD)')
out_.plot.kde(ax=ax, legend=True)
plt.xlim(-0.05, 0.08)
ax.grid(axis='x')
ax.grid(axis='y')

In [7]:
labels[np.where(labels!=args.normal_class)[0]] = 20
labels[np.where(labels==args.normal_class)[0]] = 10
labels[np.where(labels==10)[0]] = 0
labels[np.where(labels==20)[0]] = 1

In [8]:
from sklearn.metrics import roc_auc_score

test_auc = roc_auc_score(labels, scores)
test_auc

0.8381346710866859